In [1]:
%pip install cvlib


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import os
import cv2
import numpy as np
import cvlib as cv

In [3]:
# Load model
model = load_model('gender_model.h5')

In [4]:
#open webcam
webcam = cv2.VideoCapture(1)

classes = ['man', 'woman']

In [5]:
# Initialize variables
face_present = False
no_face_count = 0  # Counter to track consecutive frames with no detected face
no_face_threshold = 30  # Threshold for determining if the face has left the screen

while webcam.isOpened():
    # Read frame from webcam
    status, frame = webcam.read()
    if not status:
        break  # Break the loop if no frame is read

    # Apply face detection
    face, confidence = cv.detect_face(frame)

    # Check if no faces are detected
    if len(face) == 0:
        no_face_count += 1
        if face_present and no_face_count > no_face_threshold:
            print("Face has left the frame.")
            face_present = False  # Reset face_present
            no_face_count = 0  # Reset the counter
        
        # Display feedback on the frame
        cv2.putText(frame, "Face not detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('gender', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    # Reset no_face_count when a face is detected
    no_face_count = 0
    face_present = True  # Update face_present to True

    # Loop through detected faces
    for idx, f in enumerate(face):
        print(f"f at index {idx}: {f}")  # Debugging line

        # If f is not in the expected format, skip it
        if len(f) < 4:
            print(f"Skipping invalid face: {f}")
            continue

        # Get corner points of face rectangle
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # Draw rectangle over face
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 255), 2)

        # Crop the detected face region
        face_crop = np.copy(frame[startY:endY, startX:endX])

        if face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
            continue

        # Preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (120, 120))
        face_crop = face_crop.astype('float32') / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # Apply gender detection on face
        conf = model.predict(face_crop)[0]

        # Get label with max accuracy
        idx = np.argmax(conf)
        label = classes[idx]
        label = '{}: {:.2f}%'.format(label, conf[idx] * 100)
        Y = startY - 10 if startY - 10 > 10 else startY + 10

        cv2.putText(frame, label, (startX, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    # Display output
    cv2.imshow('gender', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

f at index 0: [np.int64(175), np.int64(196), np.int64(389), np.int64(457)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
f at index 0: [np.int64(175), np.int64(196), np.int64(388), np.int64(459)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
f at index 0: [np.int64(176), np.int64(198), np.int64(388), np.int64(455)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
f at index 0: [np.int64(176), np.int64(198), np.int64(389), np.int64(457)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
f at index 0: [np.int64(176), np.int64(196), np.int64(389), np.int64(457)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
f at index 0: [np.int64(176), np.int64(197), np.int64(389), np.int64(457)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
f at index 0: [np.int64(177), np.int64(193), np.int64(387), np.int64(456)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
f at index 0: [np.int64(178), np.int64(197), np.int64(388), np.int64(454)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
f at index 0: [np.int64(176), np.int64(193), np.int64(388), np.int64(456)]
1/1 ━━━━━━━━━━━━━━━━